In [ ]:
#dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
import pymongo

In [ ]:
# Initialize PyMongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.mars_db
collection = db.items

# Headlines

In [ ]:
#url and get page with requests
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [ ]:
#get your soup (object)
headlines_soup = bs(response.text, 'lxml')

In [ ]:
#scrape the latest news headline text and get rid of the white space
headline = headlines_soup.find('div', class_ = 'content_title').text.strip()

In [ ]:
#print it to make sure it works
print(headline)

In [ ]:
#scrape the blurb text that goes with the headline - a.k.a. the first paragraph without white space 
news = headlines_soup.find('div', class_ = 'rollover_description_inner').text.strip()

In [ ]:
#print to make sure that one works
print(news)

# Display the featured Mars Image

In [ ]:
# Import Selenium & Splinter and set the chromedriver path (I'm on windows)
from selenium import webdriver
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [ ]:
#visit the URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [ ]:
#click the web button to get to the full image
browser.click_link_by_id('full_image')

In [ ]:
#hold for two seconds to let it load
time.sleep(2)

In [ ]:
#scrape browser into soup and find full resolution image of Mars
#Call up the image
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [ ]:
#append the url snippet to make it the full url of the image 
featured_img_url = "https://www.jpl.nasa.gov"

In [ ]:
#verify we have the url
featured_img_url

# Mars Weather Tweet

In [ ]:
#on to getting the latest tweet about the weather on Mars
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)

In [ ]:
# create soup object
weather_soup = bs(response.text, 'html.parser')

In [ ]:
#pull all the tweets in case the first one isn't the one you want
mars_weather_tweet = weather_soup.select(".tweet-text")

In [ ]:
print(mars_weather_tweet)

In [ ]:
[x.text for x in mars_weather_tweet]

In [ ]:
#verify you got the right info 
print(mars_weather_tweet)


# Facts

In [ ]:
#visit the space facts site
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [ ]:
# find the html table and convert to pandas dataframe
mars_df = pd.read_html(url)
mars_df = (mars_df[0])

In [ ]:
#set column headers 
mars_df.columns = ["Description", "Value"]
# mars_df = mars_df.set_index("Description")


In [ ]:
mars_df

In [ ]:
mars_df = mars_df.to_html(classes='mars')
table_data = mars_df.replace('\n', ' ')
table_data

# database

In [ ]:
#dictionary to be inserted as a MongoDB document
post = {
    'headline': headline, 
    'news': news, 
    'featured': featured_img_url,
    'weather': mars_weather_tweet,
    'facts': table_data
}
collection.insert_one(post)